In [34]:
import numpy as np
import pickle
from glob import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange


In [2]:
def path_to_indseq(table, path, count_table):
    indseqs = []
    countseqs = []
    videos = sorted(glob('{}/*'.format(path)))
    for video in videos:
        frames = sorted(glob(video + '/*'))
        
        frame_ind = [int(frame.split('frame')[-1].split('.')[0]) for frame in frames]
        order = np.argsort(frame_ind)
        ordered_frames = np.array(frames)[order]
        
        #get sequence of counts
        count_seq = [count_table[frame.split('/')[-1]] for frame in ordered_frames]
        countseqs.append(count_seq)
        ind_seq = [table[frame.split('/')[-1]] for frame in ordered_frames]
        indseqs.append(ind_seq)
    return indseqs, countseqs

In [3]:
with open('/home/beckmann/path_to_count_table.pkl', 'rb') as f:
    rawdata = f.read()
path_to_count_table = pickle.loads(rawdata)

with open('/home/beckmann/imnet_10k/table.pkl', 'rb') as f:
    rawdata = f.read()
imnet_table = pickle.loads(rawdata)

path = '/mnt/storage02/tamedia/keyframes/nn_keyframes/'
videos = sorted(glob('{}/*'.format(path)))

bpe_data, bpe_counts = path_to_indseq(imnet_table, path, path_to_count_table)

In [73]:
class our_lstm(nn.Module):
    def __init__(self, vocabSz, hiddenSz):
        super(our_lstm, self).__init__()
        self.emb = nn.Embedding(vocabSz, hiddenSz)
        self.dec = nn.LSTM(hiddenSz, hiddenSz)
        self.lin = nn.Linear(hiddenSz, vocabSz)
    def forward(self, input):
        embeddings = self.emb(input)
        out, hidden = self.dec(embeddings)
        out = self.lin(out.view(-1, out.size(2)))
        return F.softmax(out, dim=1)

In [93]:
device = torch.device('cuda:2')
model = our_lstm(10000, 128).cuda()
#model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs=10

#for epoch in tqdm(range(epochs)):
#for indseq, counts in (zip(bpe_data, bpe_counts)):
#loss_list=[]
for epoch in range(epochs):
    print('Epoch {}.'.format(epoch))
    t = trange(len(bpe_data))
    for i in t:
        indseq = bpe_data[i]
        if len(indseq)>1:
            # to torch tensor
            indseq_tensor = torch.tensor([indseq[:-1]])

            # Step 1. Clear gradients.
            model.zero_grad()

            # Step 2. Run our forward pass.
            scores = model(indseq_tensor.cuda())

            # Step 3. Compute the loss, gradients, and update the parameters by
            loss = loss_function(scores, torch.tensor(indseq[1:]).cuda())
            #loss_list.append(loss.item())
            t.set_postfix(loss=loss.item())
            loss.backward()
            optimizer.step()
            
    print('Final loss: {}.'.format(loss.item()))

Epoch 0.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed


Final loss: 9.210512161254883.
Epoch 7.


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [90]:
torch.max(model.forward(torch.tensor([bpe_data[0]]).cuda()), dim=1)

torch.return_types.max(
values=tensor([0.9999, 0.9979, 0.8129, 0.5327, 0.9377, 1.0000, 0.9807],
       device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([9066, 9066, 9066, 9066, 9066, 9066, 9066], device='cuda:0'))

In [91]:
bpe_data[0]

[1520, 8783, 5407, 724, 8494, 1224, 1773]